In [ ]:
import pandas as pd
import torch 
import numpy as np
from torch.utils.data import Dataset, DataLoader
import sys
sys.path.append("..")
from utils import pad_collate
from dataloader_comma import CommaDataset
from collections import Counter
from model import VTN
import matplotlib.pyplot as plt 
from PIL import Image
import glob
import os
from utils import * 
import re
from vis_utils import * 
from scipy.signal import find_peaks

### Dataloader Creation

In [ ]:
from torch.utils.data import DataLoader
from dataloader_comma import CommaDataset

# Create the dataset
dataset_comma = CommaDataset(
    dataset_type="test",  # o "val" o "train" depends on what we want plot
    use_transform=False,
    multitask="distance",  
    ground_truth="desired", # Change to True to have all the parameters you use in the plot
    dataset_path="/kaggle/input/final-hdf5-files",
    dataset_fraction=1.0
)

print(f"Dataset created with {len(dataset_comma)} samples")

# Creating the Dataloader
dataloader_comma = DataLoader(
    dataset_comma,
    batch_size=1,  # Keep 1 for plotting
    shuffle=False,  # Don't shuffle for plots
    num_workers=0,  # 0 for easier debugging
    # collate_fn=None  # Use the default collate_fn, NOT the custom one you showed
)

print("DataLoader created successfully")

In [ ]:
all_f = []
for j, batch in enumerate(dataloader_comma):
    img, vego, angle, distance, gas, brake, ccenabled, seq_key = batch

    intervention = ~ccenabled.squeeze() | brake.squeeze() | gas.squeeze()

    img, angle, distance, vego = img.to(gpu), angle.to(gpu), distance.to(gpu), vego.to(gpu)
    (logits, attns), concepts = model(img, angle, distance, vego, seq_key)
    img, vego, angle, distance, gas, brake, ccenabled = img, vego.squeeze(), angle.squeeze(), distance.squeeze(), gas.squeeze(), brake.squeeze(), ccenabled.squeeze()
    angle, distance, vego, logits = angle.to("cpu"), distance.to("cpu"), vego.to("cpu"), logits.detach().cpu().to("cpu")

    atten = attns[0][:,:,0:concepts.shape[1]].detach()
    seq_len = atten.shape[2]
    alignment_array = get_aligned_attention(atten.squeeze().cpu(), seq_len)
    speed_graph_0 = alignment_array.sum(axis=0)[8:-8]
    speed_graph_0 = moving_average(speed_graph_0, 10)

    atten = attns[1][:,:,0:concepts.shape[1]].detach()
    seq_len = atten.shape[2]
    alignment_array = get_aligned_attention(atten.squeeze().cpu(), seq_len)
    speed_graph_1 = alignment_array.sum(axis=0)[8:-8]
    speed_graph_1 = moving_average(speed_graph_1, 10)

    atten = attns[2][:,:,0:concepts.shape[1]].detach()
    seq_len = atten.shape[2]
    alignment_array = get_aligned_attention(atten.squeeze().cpu(), seq_len)
    speed_graph_2 = alignment_array.sum(axis=0)[8:-8]
    speed_graph_2 = moving_average(speed_graph_2, 10)
    
    f = []
    inter = []
    m = (speed_graph_2 + speed_graph_1 + speed_graph_0)
    plt.plot(speed_graph_2, color="black")
    plt.plot(speed_graph_1, color ="blue")
    plt.plot(speed_graph_0, color ="red")
    ali = 1 + m 
    concepts = concepts.squeeze()
    weighted_concepts = concepts.cpu() * ali.reshape((ali.shape[0],1))
   
    smoothed_signal = np.convolve(m, np.ones(8) / 8, mode='same')
    peaks, _ = find_peaks(smoothed_signal, distance=16)

    # Plot original and smoothed signals
    plt.plot(peaks, smoothed_signal[peaks], 'ro', label='Peaks')
    plt.plot(smoothed_signal, 'r-')
    binary_array = np.zeros(len(m), dtype=np.int64)
    


    for elem in peaks:
        binary_array[max(0, elem-4): min(len(m), elem+4)] = 1
    binary_array = (smoothed_signal-smoothed_signal.mean()) < 0.1
    attention_concepts = concepts.squeeze()[binary_array == 1, :]
    top5_indices = torch.tensor(attention_concepts).topk(5).indices
    for i, elem0 in enumerate(top5_indices):
        for a in elem0: 
            f.append(scenarios)
    all_f.extend(f)

### Print the top N concepts

In [10]:
flattened = []
for f in all_f: 
    flattened.extend(f)

In [11]:
a = np.unique(np.array(flattened))
c = Counter(np.array(flattened))

In [ ]:
mc = c.most_common(6)
mc

In [ ]:
mc = c.most_common(6)
mc

In [ ]:
for j, batch in enumerate(dataloader_comma):
    img, vego, angle, distance, gas, brake, ccenabled, seq_key = batch

    img, angle, distance, vego = img.to(gpu), angle.to(gpu), distance.to(gpu), vego.to(gpu)
    (logits, attns), concepts = model(img, angle, distance, vego, seq_key)
    img, vego, angle, distance, gas, brake, ccenabled = img, vego.squeeze(), angle.squeeze(), distance.squeeze(), gas.squeeze(), brake.squeeze(), ccenabled.squeeze()
    logits, attns = logits.squeeze(), attns.squeeze()
    attns = attns[ccenabled]
    angle, distance, vego, logits = angle.to("cpu"), distance.to("cpu"), vego.to("cpu"), logits.detach().cpu().to("cpu")

    concepts
    atten = attns[0][:,:,0:concepts.shape[1]].detach()
    seq_len = atten.shape[2]
    alignment_array = get_aligned_attention(atten.squeeze().cpu(), seq_len)
    speed_graph_0 = alignment_array.sum(axis=0)[8:-8]
    speed_graph_0 = moving_average(speed_graph_0, 10)

    atten = attns[1][:,:,0:concepts.shape[1]].detach()
    seq_len = atten.shape[2]
    alignment_array = get_aligned_attention(atten.squeeze().cpu(), seq_len)
    speed_graph_1 = alignment_array.sum(axis=0)[8:-8]
    speed_graph_1 = moving_average(speed_graph_1, 10)

    atten = attns[2][:,:,0:concepts.shape[1]].detach()
    seq_len = atten.shape[2]
    alignment_array = get_aligned_attention(atten.squeeze().cpu(), seq_len)
    speed_graph_2 = alignment_array.sum(axis=0)[8:-8]
    speed_graph_2 = moving_average(speed_graph_2, 10)
    
    f = []
    inter = []
    m = (speed_graph_2 + speed_graph_1 + speed_graph_0)/3
    ali = 1 + m 
    print(ali.shape)
    concepts = concepts.squeeze()
    weighted_concepts = concepts * ali

    top5_indices = torch.tensor(concepts).topk(5).indices
    
    for i, elem0 in enumerate(top5_indices):
        inter = []
        for elem in top5_indices[max(i-5, 0):min(i+5,len(top5_indices))]:
            l = elem.cpu().numpy().tolist()
            if 131 in l:
                l.remove(131)
            inter.extend(l)
        count_dict = Counter(inter)
        # Get the top 5 most occurring numbers
        top_5 = count_dict.most_common(2)
        intermediate = []
        for a in top_5: 
            intermediate.append(scenarios[a[0]])
        f.append(intermediate)

    normalized_weights = atten#np.array(attention_weights) / np.sum(attention_weights)
    print(atten.shape)
    

    fig, ax = plt.subplots(figsize=(10, 6))
    plt.plot(speed_graph_0, label="1")
    plt.plot(speed_graph_1, label="2")
    plt.plot(speed_graph_2, label="3")
    plt.plot(m, label="mean")
    plt.plot(alignment_array[0:70, :].sum(axis=0), label="align")
    plt.legend()
    #heatmap = ax.pcolormesh(alignment_array[0:70, :], cmap='hot')
    # Add colorbar
    #cbar = plt.colorbar(heatmap)

    # Add labels and title
    ax.set_xlabel('Sequence Position')
    ax.set_ylabel('Window')
    ax.set_title('Longformer Sliding Chunk Attention')
    plt.show()
    # Show the plot
    #plt.savefig(f"/kaggle/working/attention_vis{j}.pdf")
    #plt.close()
    break

In [3]:
plt.rcParams.update({'font.size': 26}) 